# Spam Detection

- we'll use the Naive Bayes Intuiton Technique for Spam Detection

## Naive Bayes Intuition
- assume that inputs are independent from each other
- this is what makes it naive?
- what kind of distribution to pick? 
- totally up to you
- you can also pick different distributions for different inputs
- ex: X1 Gaussian, x2 Exponential, x3 Bernoulli
- In Sklearn we have predefined models, all inputs come from the same kind of distribution (Gaussian, Multinomial, Bernoulli): you cannot choose different distributions for your Xs
- how do you choose? #by looking t your data
- if your data is continuos and looks like a bell curve: Gaussian, if categorical: multinomial. the latter is typically the correct option for NLP, specifically for count vectors and Tf-idf
- why tfidf de good option? anlamadım
- if data is binary bernoulli would be good choice, this may be applicable if you choose a binary version of count vector sfor nlp

## Exercise Prompt
- a dataset of sms, labeled spam and non spam
- build a classifier, split into train and test dataset, assess accuracy

### Details
- vectoirizng strategy of your choice
- options:  tokenization, lemmatization, normalization
- classifier: an appropriate form of Naive Bayes, either from sklearn or if you're advanced build yourself
- feel free to try other classifiers
- score function returns accuracy
- not ideal when classes are imbalanced, 
- if imbalanced check other metrics function like AUC / F1-score

In [1]:
# data from:
# https://www.kaggle.com/uciml/sms-spam-collection-dataset
!wget https://lazyprogrammer.me/course_files/spam.csv

--2023-06-14 00:07:03--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: 'spam.csv.2'

spam.csv.2          100%[===================>] 491.86K   505KB/s    in 1.0s    

2023-06-14 00:07:08 (505 KB/s) - 'spam.csv.2' saved [503663/503663]



In [2]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import nltk #kirill'de bunla yaptık
from nltk.stem.porter import PorterStemmer #kirill
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB #kirill'le olan positive negative sentiment ama Gaussian kullanmış
#bu dersin hocası Multinomial kullanıyo :/
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.metrics import accuracy_score #kirill

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yagmuraslan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
df = pd.read_csv("spam.csv")
df.head()

#excelde comma-delimited ufc-8 olarak kaydettikten sonra

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df = df[["v1","v2"]]
df["v2"]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ã_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [6]:
texts = df["v2"]
labels = df["v1"]
type(labels)
labels[2]

'spam'

In [7]:
labels = (labels == "spam")
labels = labels.astype(int)

In [8]:
text_train, text_test, labels_train, labels_test = train_test_split(texts, labels)

In [9]:
type(text_train)

pandas.core.series.Series

### Önce Word_tokenize kullanak istedim çünkü bence punctuation ve special karakterler sonucu etkileyebilir

In [8]:
vectorizer = CountVectorizer(tokenizer = word_tokenize)

In [9]:
Xtrain = vectorizer.fit_transform(text_train)
Xtest = vectorizer.transform(text_test)

In [10]:
classifier = MultinomialNB()
classifier.fit(Xtrain, labels_train)

MultinomialNB()

In [11]:
print("train score:", classifier.score(Xtrain, labels_train))
print("test score:", classifier.score(Xtest, labels_test))

train score: 0.9932998324958124
test score: 0.9791816223977028


In [12]:
labels_pred = classifier.predict(Xtest)

In [13]:
cm = confusion_matrix(labels_test, labels_pred)
print(cm)
accuracy_score(labels_test, labels_pred) 

[[1203    7]
 [  22  161]]


0.9791816223977028

- train score: 0.9932998324958124
- test score: 0.9791816223977028

### Şimdi count vectorizer'ın default tokenizerı ile deneyelim:

In [21]:
vectorizer = CountVectorizer()
Xtrain = vectorizer.fit_transform(text_train)
Xtest = vectorizer.transform(text_test)

classifier = MultinomialNB()
classifier.fit(Xtrain, labels_train)

print("train score:", classifier.score(Xtrain, labels_train))
print("test score:", classifier.score(Xtest, labels_test))

train score: 0.9944962909787031
test score: 0.9842067480258435


In [15]:
labels_pred = classifier.predict(Xtest)
cm = confusion_matrix(labels_test, labels_pred)
print(cm)
accuracy_score(labels_test, labels_pred) 

[[1203    7]
 [  15  168]]


0.9842067480258435

- train score: 0.9944962909787031
- test score: 0.9842067480258435

### Şimdi Tf-idf vectorizer ile deneyelim

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000)
Xtrain = vectorizer.fit_transform(text_train)
Xtest = vectorizer.transform(text_test)

classifier = MultinomialNB()
classifier.fit(Xtrain, labels_train)

print("train score:", classifier.score(Xtrain, labels_train))
print("test score:", classifier.score(Xtest, labels_test))

train score: 0.9827709978463748
test score: 0.9741564967695621


In [18]:
labels_pred = classifier.predict(Xtest)
cm = confusion_matrix(labels_test, labels_pred)
print(cm)
accuracy_score(labels_test, labels_pred) 

[[1209    1]
 [  35  148]]


0.9741564967695621

### şimdi countvectorizer'ı stopwords ile deneyelim

In [22]:
vectorizer = CountVectorizer(stop_words = "english")
Xtrain = vectorizer.fit_transform(text_train)
Xtest = vectorizer.transform(text_test)

classifier = MultinomialNB()
classifier.fit(Xtrain, labels_train)

print("train score:", classifier.score(Xtrain, labels_train))
print("test score:", classifier.score(Xtest, labels_test))

train score: 0.9944962909787031
test score: 0.9820531227566404


In [23]:
## default count vectorizer ile hiçbir fark yaratamadı

### şimdi countvectorizer'ı  lemmatization ile deneyelim

In [34]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet

In [35]:
nltk.download("wordnet")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yagmuraslan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yagmuraslan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yagmuraslan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/yagmuraslan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [36]:
def get_wordnet_pos(treebank_tag):
  if treebank_tag.startswith('J'):
    return wordnet.ADJ
  elif treebank_tag.startswith('V'):
    return wordnet.VERB
  elif treebank_tag.startswith('N'):
    return wordnet.NOUN
  elif treebank_tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [37]:
class LemmaTokenizer:
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, doc):
    tokens = word_tokenize(doc) #this converts doc to tokens, like a fancier version of string split
    words_and_tags = nltk.pos_tag(tokens) #this obtains pos tags, returns a tuple
    return [self.wnl.lemmatize(word, pos=get_wordnet_pos(tag)) \
            for word, tag in words_and_tags]


In [38]:
vectorizer = CountVectorizer(tokenizer= LemmaTokenizer())
Xtrain = vectorizer.fit_transform(text_train)
Xtest = vectorizer.transform(text_test)

classifier = MultinomialNB()
classifier.fit(Xtrain, labels_train)

print("train score:", classifier.score(Xtrain, labels_train))
print("test score:", classifier.score(Xtest, labels_test))

train score: 0.992581957406078
test score: 0.9784637473079684


### şimdi de stemmer ile deneyelim

In [43]:
class StemTokenizer:
    def __init__(self):
        self.porter = PorterStemmer()
    def __call__(self,doc):
        tokens = word_tokenize(doc)
        return(self.porter.stem(t) for t in tokens)

In [44]:
vectorizer = CountVectorizer(tokenizer= StemTokenizer())
Xtrain = vectorizer.fit_transform(text_train)
Xtest = vectorizer.transform(text_test)

classifier = MultinomialNB()
classifier.fit(Xtrain, labels_train)

print("train score:", classifier.score(Xtrain, labels_train))
print("test score:", classifier.score(Xtest, labels_test))

train score: 0.9932998324958124
test score: 0.9784637473079684


#### en iyi sonucu sanki count vectorizer'ın default'u ile aldım.
## Bi de Gaussian olsa nolur acaba bakalım

In [45]:
vectorizer = CountVectorizer(stop_words = "english")
Xtrain = vectorizer.fit_transform(text_train)
Xtest = vectorizer.transform(text_test)

classifier = GaussianNB()
classifier.fit(Xtrain, labels_train)

print("train score:", classifier.score(Xtrain, labels_train))
print("test score:", classifier.score(Xtest, labels_test))

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [47]:
vectorizer = CountVectorizer(stop_words = "english")
Xtrain = vectorizer.fit_transform(text_train).toarray()
Xtest = vectorizer.transform(text_test).toarray()

classifier = GaussianNB()
classifier.fit(Xtrain, labels_train)

print("train score:", classifier.score(Xtrain, labels_train))
print("test score:", classifier.score(Xtest, labels_test))

### BAYA KÖTÜ

train score: 0.9373055754965303
test score: 0.8894472361809045


### AY ÇOK GARİP Bİ ŞEY FARK ETTİMMMMM!!!! 
## KİRİL TRAİN TEST SPLIT'TEN ÖNCE YAPIYOR VECTORIZATION'I BİZSE SONRA YAPIYORUZ, ARTILARI EKSİLERİ NELERDİR?

- Gaussian Naive Bayes (based on the Gaussian distribution) is suitable when dealing with continuous numerical features that follow a Gaussian (normal) distribution. If your input features are continuous variables (e.g., word frequencies) and exhibit a roughly Gaussian distribution, Gaussian Naive Bayes can be a reasonable choice.

- On the other hand, if your input features are discrete or represent counts (e.g., word occurrences), Multinomial Naive Bayes is more appropriate. Multinomial Naive Bayes assumes that features follow a multinomial distribution, which is often the case with text data or categorical variables.

- In the context of spam detection, text data is commonly used, and Multinomial Naive Bayes is a popular choice. It allows you to represent emails as vectors of word frequencies or presence indicators, making it well-suited for text classification tasks.

- Therefore, for spam detection using a Naive Bayes classifier, the preferred choice would typically be the Multinomial Naive Bayes classifier. However, it is recommended to experiment and evaluate both Gaussian and Multinomial Naive Bayes classifiers on your specific dataset to determine which one performs better in terms of accuracy and meets your specific requirements.

#### Bernoulli Distribution
- Certainly! One example where you may want to use the Bernoulli distribution with a Naive Bayes classifier is in sentiment analysis or text classification tasks where the focus is on the presence or absence of specific features (words or binary indicators) in the text.

- For instance, consider a sentiment analysis task where you want to classify movie reviews as positive or negative based on the presence of certain words in the review. In this case, you can represent the input data as a binary matrix, where each row corresponds to a movie review and each column represents the presence or absence of a specific word.

- The Bernoulli Naive Bayes classifier is well-suited for this scenario because it assumes that the features are binary variables (e.g., word presence indicators) and models the conditional probabilities of the features given the class label. It calculates the likelihood of a feature being present or absent in each class and combines this information with the prior probabilities of the classes to make predictions.